# Rainfall erosivity
The rainfall erosivity is the erosive force of rainfall. 



## How to estimate the rainfall erosivity?
The R factor is typically expressed in units of MJ·mm·ha⁻¹·h⁻¹·yr⁻¹ (metric units) and is based on the product of storm kinetic energy (E) and the maximum 30-minute rainfall intensity (I₃₀). Estimating the rainfall erosivity relies on high-resolution rainfall intensity data as direct calculation of E and I₃₀ requires sub-hourly rainfall data (e.g., 15- or 30-minute intervals).

### Considerations:
- **Regional Variations**: The relationship between rainfall depth and erosivity varies significantly by region due to differences in rainfall patterns, raindrop size, and other factors. Therefore, it's crucial to use methods and equations that are appropriate for the specific location.
- **Data Quality**: The accuracy of the R-factor estimation depends on the quality and reliability of the precipitation data.
- **Long-Term Averages**: The RUSLE R-factor represents a long-term average, so it's essential to use data from a sufficiently long period (e.g., 20 years or more) to obtain a reliable estimate.

Here's a step-by-step implementation to calculate the R-factor from 10-minute rainfall data in an Excel file, following the RUSLE methodology

In [1]:
import pandas as pd
import numpy as np

# Replace with your file path
file_path = 'rainfall_data.xlsx'  

# Read data with explicit column names
rainfall_data = pd.read_excel(
    file_path,
    usecols=['Date', 'Time', 'Rainfall'],  # Ensure columns match your Excel file
    parse_dates={'DateTime': ['Date', 'Time']},  # Combine date/time
    infer_datetime_format=True)

# Sort by time and remove rows with missing rainfall
rainfall_data = rainfall_data.sort_values('DateTime').dropna(subset=['Rainfall'])

# Criteria to Identify rain Events: Events separated by ≥ time_between_events with rainfall > rain_threshold
time_between_events = 1 # hours
rain_threshold = 1 # mm
rainfall_data['TimeDiff'] = rainfall_data['DateTime'].diff().dt.total_seconds() / 3600
rainfall_data['Event'] = (rainfall_data['TimeDiff'] > time_between_events) | (rainfall_data['Rainfall_mm'] >= rain_threshold)
rainfall_data['Event'] = rainfall_data['Event'].cumsum()  # Assign event IDs

# Calculate EI30 for Each Event
def calculate_ei30(event_group):
    # Kinetic energy calculation
    event_group['I10'] = event_group['Rainfall_mm'] / (10/60)  # mm/h (10-min intensity)
    event_group['Kinetic_Energy'] = 0.119 + 0.0873 * np.log(event_group['I10'].replace(0, np.nan))
    event_group['Energy'] = event_group['Kinetic_Energy'] * event_group['Rainfall_mm']
    
    # Max 30-min intensity (3 consecutive 10-min intervals)
    event_group['I30'] = event_group['I10'].rolling(3, min_periods=1).sum().max()
    
    # Total energy for the event
    total_energy = event_group['Energy'].sum()
    
    return total_energy * event_group['I30'].max()

# Apply to all events
erosive_events = rainfall_data.groupby('Event').apply(calculate_ei30).rename('EI30')

# Identify Erosive Events
# Apply threshold: Total rainfall ≥ erosive_threshold per event
erosive_threshold = 10 # mm
event_totals = rainfall_data.groupby('Event')['Rainfall_mm'].sum()
valid_events = event_totals[event_totals >= erosive_threshold].index
erosive_events = erosive_events[erosive_events.index.isin(valid_events)]

# Compute the R-factor
# Sum EI30 values annually
rainfall_data['Year'] = rainfall_data['DateTime'].dt.year
annual_r_factor = erosive_events.groupby(rainfall_data['Year']).sum()

# Average over years
mean_annual_r = annual_r_factor.mean()
print(f"Mean Annual R-Factor: {mean_annual_r:.2f} MJ·mm·ha⁻¹·h⁻¹·yr⁻¹")


TypeError: read_excel() got an unexpected keyword argument 'infer_datetime_format'

## What if we only have daily precipitation data available?
Estimating the rainfall erosivity using daily precipitation data can be complex, as the ideal calculation relies on high-resolution rainfall intensity data. 

Daily precipitation data lacks the necessary intensity information. This means we must rely on empirical relationships and approximations. The Modified Fournier Index and empirical equations are common approaches.

### a. Modified Fournier Index (MFI)
- This index uses monthly and annual rainfall data to estimate erosivity.   
- It's a common approach when detailed intensity data is unavailable.   
- Equations relating MFI to the R-factor have been developed for various regions.
- This method is very useful when only daily total rainfall amounts are available. By summing the daily amounts, monthly, and annual amounts can be found.

\$
F = \frac{\sum_{i=1}^{12} P_i^2}{P_{\text{total}}}
\$

Where:
- $P_i$ = total precipitation in month \(i\) (mm), derived from summing daily data for each month.
- $P_{\text{total}}$ = total annual precipitation (mm).

Then, relate $F$ to $R$ using a regional regression equation, such as:
$
R = a \cdot F^b
$

Where $a$ and $b$ are coefficients specific to your region (you’ll need local calibration data or references from RUSLE documentation or studies in your area).

To apply this:
1. Sum daily precipitation into monthly totals ($P_i$).
2. Calculate \(F\) using the formula above.
3. Use the region-specific $a$ and $b$ values to estimate $R$.

### Python implementation
Below is a Python script that reads daily precipitation data from an Excel file, calculates the R factor using the Modified Fournier Index (MFI) method, and applies a regional regression equation. Note: We'll filter out days with precipitation < 10 mm (a common threshold for erosive events).


#### How It Works:
1. **Read Data**: Loads the Excel file into a pandas DataFrame.
2. **Filter**: Removes days with precipitation below the threshold (e.g., 10 mm).
3. **Aggregate**: Groups daily data into monthly totals by year and month.
4. **Calculate MFI**: Computes the Fournier Index ($F = \sum P_i^2 / P_{\text{total}}$) for each year.
5. **Estimate R**: Applies the equation $R = a \cdot F^b$ for each year.
6. **Average**: Computes the average R factor across all years.


In [ ]:
# Function to calculate the R factor using the Modified Fournier Index
def calculate_r_factor(file_path, precip_col='Precipitation', date_col='Date', a=0.073, b=1.4, threshold=10):
    # Step 1: Read the Excel file
    try:
        df = pd.read_excel(file_path)
        print("Excel file loaded successfully.")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return None
    
    # Step 2: Ensure the Date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Step 3: Filter out days with precipitation below the threshold
    df_filtered = df[df[precip_col] >= threshold].copy()
    
    # Step 4: Aggregate daily precipitation into monthly totals
    df_filtered['Month'] = df_filtered[date_col].dt.month
    df_filtered['Year'] = df_filtered[date_col].dt.year
    monthly_totals = df_filtered.groupby(['Year', 'Month'])[precip_col].sum().reset_index()
    
    # Step 5: Calculate the Modified Fournier Index (F) for each year
    yearly_f = []
    for year in monthly_totals['Year'].unique():
        yearly_data = monthly_totals[monthly_totals['Year'] == year]
        p_i_squared = (yearly_data[precip_col] ** 2).sum()  # Sum of monthly precipitation squared
        p_total = yearly_data[precip_col].sum()  # Annual total precipitation
        if p_total > 0:  # Avoid division by zero
            f = p_i_squared / p_total
            yearly_f.append((year, f))
        else:
            yearly_f.append((year, 0))
    
    # Step 6: Calculate R factor for each year using R = a * F^b
    r_factors = []
    for year, f in yearly_f:
        if f > 0:
            r = a * (f ** b)
            r_factors.append((year, r))
        else:
            r_factors.append((year, 0))
    
    # Step 7: Average R factor across years (if multi-year data)
    if r_factors:
        avg_r = np.mean([r for _, r in r_factors])
    else:
        avg_r = 0
    
    # Output results
    print("\nYearly R Factors:")
    for year, r in r_factors:
        print(f"Year {year}: R = {r:.2f} MJ·mm·ha⁻¹·h⁻¹·yr⁻¹")
    print(f"\nAverage R Factor: {avg_r:.2f} MJ·mm·ha⁻¹·h⁻¹·yr⁻¹")
    
    return r_factors, avg_r

# Example usage
if __name__ == "__main__":
    # Specify the path to your Excel file
    file_path = "daily_precipitation.xlsx"  # Replace with your file path
    
    # Parameters (adjust these based on your region)
    a_coeff = 0.073  # Example coefficient
    b_exp = 1.4      # Example exponent
    erosive_threshold = 10  # Minimum precipitation (mm) for erosive events
    
    # Run the calculation
    r_factors, avg_r = calculate_r_factor(
        file_path,
        precip_col='Precipitation',  # Column name for precipitation
        date_col='Date',            # Column name for dates
        a=a_coeff,
        b=b_exp,
        threshold=erosive_threshold
    )


### Limitations
- Daily data underestimates R compared to high-resolution (e.g., 30-minute) data because it misses peak intensities.
- Accuracy improves with region-specific equations and longer datasets.


### b. Empirical Equations Based on Rainfall Depth
- Researchers have developed region-specific equations that relate daily or monthly rainfall depth to the R-factor.
- These equations often involve statistical relationships derived from observed rainfall and erosion data.
- These equations vary greatly, and it is very important to use an equation that has been developed for the region being studied.